In [1]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,Input
import os
import pandas as pd
import numpy as np

In [2]:
train_dir = 'images/train'
test_dir = 'images/test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, 'completed :)')
    return image_paths, labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(train_dir)

angry completed :)
disgust completed :)
fear completed :)
happy completed :)
neutral completed :)
sad completed :)
surprise completed :)


In [5]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(test_dir)

angry completed :)
disgust completed :)
fear completed :)
happy completed :)
neutral completed :)
sad completed :)
surprise completed :)


In [6]:
from tqdm.notebook import tqdm

In [7]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode = 'grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [8]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [9]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [10]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [11]:
x_train = train_features/255.0
x_test = test_features/255.0

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [14]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [15]:
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)

In [22]:
model = Sequential()

#convolutional layers
model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu', input_shape = (48,48,1), padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

#fully connected layers
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))

#output layer
model.add(Dense(7, activation = 'softmax'))

In [24]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [25]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs = 100, validation_data = (x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 512s 2s/step - accuracy: 0.2259 - loss: 1.8461 - val_accuracy: 0.2583 - val_loss: 1.8097
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 374s 2s/step - accuracy: 0.2506 - loss: 1.7992 - val_accuracy: 0.2768 - val_loss: 1.7750
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 391s 2s/step - accuracy: 0.2714 - loss: 1.7728 - val_accuracy: 0.3275 - val_loss: 1.6860
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 389s 2s/step - accuracy: 0.3329 - loss: 1.6692 - val_accuracy: 0.4244 - val_loss: 1.4834
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.4068 - loss: 1.5401 - val_accuracy: 0.4789 - val_loss: 1.3677
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 409s 2s/step - accuracy: 0.4334 - loss: 1.4585 - val_accuracy: 0.4883 - val_loss: 1.3337
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - accuracy: 0.4535 - loss: 1.4207 - val_accuracy: 0.5014 - val_loss: 1.2842
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 397s 2s/step - accuracy: 0.4737 - loss: 1.3615 - 

KeyboardInterrupt: 

In [29]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save('emotiondetector.keras')

In [30]:
from keras.models import model_from_json

In [32]:
json_file = open('emotiondetector.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.keras")

C:\Users\Rajiv Chaitanya M\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [34]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [35]:
def ef(image):
    img = load.img(imag, grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature/255.-0